In [13]:
import sys
import os

# Add project root to Python path
sys.path.append(os.path.abspath(".."))

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
import pickle

with open('../models/faq_data.pkl', 'rb') as f:
    faq_df = pickle.load(f)

faq_df.head()

,question,answer,clean_question,clean_answer
0,@sprintcare I have sent several private messag...,@115712 I understand. I would like to assist y...,i have sent several private messages and no on...,i understand i would like to assist you we wou...
1,@sprintcare I did.,@115712 Please send us a Private Message so th...,i did,please send us a private message so that we ca...
2,@sprintcare is the worst customer service,@115712 Can you please send us a private messa...,is the worst customer service,can you please send us a private message so th...
3,@sprintcare is the worst customer service,@115712 I would love the chance to review the ...,is the worst customer service,i would love the chance to review the account ...
4,@sprintcare is the worst customer service,@115712 Hello! We never like our customers to ...,is the worst customer service,hello we never like our customers to feel like...


In [3]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)        # remove links
    text = re.sub(r"@\w+", "", text)           # remove @mentions
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text) # remove punctuation
    text = re.sub(r"\s+", " ", text).strip()   # remove extra spaces
    return text

In [4]:
faq_df['clean_question'] = faq_df['question'].apply(clean_text)
faq_df['clean_answer'] = faq_df['answer'].apply(clean_text)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features=50000)

tfidf_matrix = vectorizer.fit_transform(faq_df['clean_question'])
tfidf_matrix.shape

(1261730, 50000)

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_response(user_query):
    user_query_clean = clean_text(user_query)
    
    user_vec = vectorizer.transform([user_query_clean])
    
    similarities = cosine_similarity(user_vec, tfidf_matrix).flatten()
    
    best_index = np.argmax(similarities)
    best_score = similarities[best_index]
    
    if best_score < 0.2:
        return "I'm not sure about that. Could you rephrase your question?"
    
    return faq_df.iloc[best_index]['answer']

In [7]:
get_response("My internet is not working")

'@230409 My apologies for the issues you are having with your internet.  Please follow/DM your service phone number and I would be happy to look into this for you. ^JK'

In [8]:
get_response("How can I reset my password?")

"@152535 Hey Marcus, help's arrived! Could you DM us your username and email address? We'll check it out 🔍 /JY https://t.co/ldFdZRiNAt"

In [9]:
get_response("I want to talk to customer care")

'@279936 You can reach out to our customer care by clicking on the link provided earlier and we will be happy to help you with this. ^KS'

In [10]:
import pickle

with open('../models/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

with open('../models/faq_data.pkl', 'wb') as f:
    pickle.dump(faq_df, f)

In [12]:
import os
os.getcwd()

'c:\\Users\\HP\\Documents\\FUTURE_ML_03\\notebooks'

In [14]:
from utils.chatbot_engine import ChatbotEngine

In [15]:
chatbot = ChatbotEngine(
    vectorizer_path='../models/vectorizer.pkl',
    faq_data_path='../models/faq_data.pkl'
)

chatbot.get_response("My internet is slow")

'@189420 Can you send me a DM so I can look into your connection? -AG'